# Regresiones 401

## Librerías y carga de datos

In [1]:
#from google.colab import drive
import numpy as np
import pandas as pd
import os
import time
import pickle
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

C:\Users\Diego\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Diego\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\Diego\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


Se monta drive en caso de que se trabaje en Colab. En otro caso, comentar celda:

In [2]:
#drive.mount('/content/drive')

In [3]:
file_path = ''#D:\Descargas\datos_gps_curso_ciencia_de_datos\'
train_path = file_path + 'train2.csv'
val_path = file_path + 'val2.csv'
test_path = file_path + 'test2.csv'

## Modelo de Regresión Lineal

#### Se crea columna de diferencia de distancia entre dos pulsos pareados.

In [4]:
train = pd.read_csv(train_path, delimiter = ',')

In [5]:
train['Distancia_Total'] = train['DistanciaInicio2'] - train['DistanciaInicio1']

In [6]:
with open('encoding1.pickle', 'rb') as handle:
    servs_encoding = pickle.load(handle)

In [7]:
train['Servicio'] = train['Servicio'].apply(lambda x: servs_encoding[x])

#### Creación del modelo y definición de hiperparámetros
En este caso se entrena con la columna de distancia y con la primera codificación de los servicios creada en codificacion_servicios.ipynb como features y con el target la columna de diferencia de tiempo.

In [8]:
X_train = train[['Distancia_Total', 'Servicio','dia_habil1','dia_Monday1','dia_Saturday1','dia_Sunday1','dia_Thursday1',
                 'dia_Tuesday1','dia_Wednesday1']]
y_train = train['Diferencia']

In [9]:
del train #se libera memoria

In [10]:
X_train = X_train.values #reshape(-1,1)
X_train.shape

(3072911, 9)

In [11]:
from sklearn.linear_model import LinearRegression

In [12]:
reg = LinearRegression()
reg.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [13]:
#se guarda el modelo entrenado para usarlo más adelante
reg_path = 'LRegressor401.sav'
pickle.dump(reg, open(reg_path, 'wb'))

Una vez entrenado el modelo, se prepara el conjunto de test para medir resultados.

## Modelo de Regresión Polinomial
Se entrena el modelo de regresión polinomial con grado 4

In [14]:
from sklearn.preprocessing import PolynomialFeatures 

In [15]:
poly = PolynomialFeatures(degree = 3) 
X_train_pol = poly.fit_transform(X_train) 

pol_reg = LinearRegression() 
pol_reg.fit(X_train_pol, y_train)

poly_path = 'polynomial401.sav'
pickle.dump(poly, open(poly_path, 'wb'))
reg_path = 'PRegressor401.sav'
pickle.dump(pol_reg, open(reg_path, 'wb'))

In [16]:
del poly #liberar memoria

Una vez entrenado el modelo, se prepara el conjunto de validación para medir resultados.

In [17]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [18]:
y_pred = reg.predict(X_train)
y_pred_pol = pol_reg.predict(X_train_pol)

In [19]:
del X_train
del X_train_pol

Resultados regresión lineal:

In [20]:
r2 = r2_score(y_train, y_pred)
mae = mean_absolute_error(y_train, y_pred)

print('R2 score: ' + str(r2) )
print('MAE score: ' + str(mae))

R2 score: 0.85296845248722
MAE score: 197.59221072275926


Resultados regresión polinomial:

In [21]:
r2 = r2_score(y_train, y_pred_pol)
mae = mean_absolute_error(y_train, y_pred_pol)

print('R2 score: ' + str(r2) )
print('MAE score: ' + str(mae))

R2 score: 0.8798901259344133
MAE score: 172.6847494552651


#### Validación

In [22]:
poly_path = 'polynomial401.sav'
pol_reg_path = 'PRegressor401.sav'
reg_path = 'LRegressor401.sav'

poly = pickle.load(open(poly_path, 'rb'))
p_reg = pickle.load(open(pol_reg_path, 'rb'))
reg = pickle.load(open(reg_path, 'rb'))

In [23]:
val = pd.read_csv(val_path, delimiter = ',')

In [24]:
val['Distancia_Total'] = val['DistanciaInicio2'] - val['DistanciaInicio1']

In [25]:
with open('encoding1.pickle', 'rb') as handle:
    servs_encoding = pickle.load(handle)

In [26]:
val['Servicio'] = val['Servicio'].apply(lambda x: servs_encoding[x])

In [27]:
val_train = val[['Distancia_Total','Servicio','dia_habil1','dia_Monday1','dia_Saturday1','dia_Sunday1','dia_Thursday1',
                 'dia_Tuesday1','dia_Wednesday1']]
val_target = val['Diferencia']

val_train = val_train.values#.reshape(-1,1)

In [31]:
y_pred = reg.predict(val_train)

y_pred_poly = p_reg.predict(poly.fit_transform(val_train))

In [33]:
r2_val = r2_score(val_target, y_pred)
mae_val = mean_absolute_error(val_target, y_pred)

print('R2 score: ' + str(r2_val))
print('MAE score: ' + str(mae_val))

R2 score: 0.8480144512425607
MAE score: 200.78462013805958


In [34]:
r2_val = r2_score(val_target, y_pred_poly)
mae_val = mean_absolute_error(val_target, y_pred_poly)

print('R2 score: ' + str(r2_val))
print('MAE score: ' + str(mae_val))

R2 score: 0.8763506148712268
MAE score: 176.17042411532756


In [48]:
def eval_range(model, dataset, poly=None, top=60, bottom=0):
    condition1 = dataset["Diferencia"] >= bottom
    condition2 = dataset["Diferencia"] < top
    total_cond = np.logical_and(condition1, condition2)
    filtered = dataset[total_cond]
    if filtered.shape[0] > 0:
        y_val = filtered["Diferencia"]
        X_val = filtered[['Distancia_Total','Servicio','dia_habil1','dia_Monday1','dia_Saturday1','dia_Sunday1','dia_Thursday1',
                 'dia_Tuesday1','dia_Wednesday1']].values
        if poly != None:
            y_pred = model.predict(poly.fit_transform(X_val))
        else:
            y_pred = model.predict(X_val)
        dif = y_val - y_pred
        rms = mean_squared_error(y_val, y_pred)
        mae = mean_absolute_error(y_val, y_pred)
        r2 = r2_score(y_val, y_pred)
        me = np.sum(dif)/(len(y_pred))
        mpe = np.sum(dif[dif > 0])/(len(y_pred))
        return rms, mae, me, mpe, r2
    else:
        return None, None, None, None, None

In [52]:
rms, mae, me, mpe, r2 = eval_range(reg, val[['Distancia_Total', 'Diferencia','Servicio','dia_habil1','dia_Monday1','dia_Saturday1','dia_Sunday1','dia_Thursday1',
                 'dia_Tuesday1','dia_Wednesday1']], top=1e20, bottom=0)
print("Errores globales")
print("\tRMS: \t{}".format(rms))
print("\tMAE: \t{}".format(mae))
print("\tME: \t{}".format(me))
print("\tMPE: \t{}".format(mpe))
print("\tR2: \t{}\n".format(r2))


ranges = [[0, 5*60], [5*60, 10*60], [10*60, 20*60], [20*60, 40*60], [40*60, 60*60]]
for bottom, top in ranges:
    rms, mae, me, mpe, r2 = eval_range(reg, val[['Distancia_Total', 'Diferencia','Servicio','dia_habil1','dia_Monday1','dia_Saturday1','dia_Sunday1','dia_Thursday1',
                 'dia_Tuesday1','dia_Wednesday1']], top=top, bottom=bottom)
    print("Errores para el rango {}min a {}min".format(bottom//60, top//60))
    print("\tRMS: \t{}".format(rms))
    print("\tMAE: \t{}".format(mae))
    print("\tME: \t{}".format(me))
    print("\tMPE: \t{}".format(mpe))
    print("\tR2: \t{}\n".format(r2))

Errores globales
	RMS: 	105951.43625830908
	MAE: 	200.78462013805958
	ME: 	-8.334836588708466
	MPE: 	96.22489177467551
	R2: 	0.8480144512425607

Errores para el rango 0min a 5min
	RMS: 	11918.462123858386
	MAE: 	90.77582081133315
	ME: 	-77.99192311042042
	MPE: 	6.391948850456358
	R2: 	-1.0629130547506067

Errores para el rango 5min a 10min
	RMS: 	26678.664739328276
	MAE: 	118.92012401770114
	ME: 	-61.1610874697211
	MPE: 	28.879518273990023
	R2: 	-2.605147738707035

Errores para el rango 10min a 20min
	RMS: 	74262.31101524924
	MAE: 	204.917078045075
	ME: 	-22.212160897949413
	MPE: 	91.35245857356277
	R2: 	-1.5104844339055559

Errores para el rango 20min a 40min
	RMS: 	243682.2597014591
	MAE: 	384.4407735866711
	ME: 	91.18171805066737
	MPE: 	237.81124581866928
	R2: 	-1.084207396372317

Errores para el rango 40min a 60min
	RMS: 	641416.2359028172
	MAE: 	649.4608435898864
	ME: 	355.4427249157072
	MPE: 	502.45178425279676
	R2: 	-4.443656759973125



In [51]:
rms, mae, me, mpe, r2 = eval_range(p_reg, val[['Distancia_Total', 'Diferencia','Servicio','dia_habil1','dia_Monday1','dia_Saturday1','dia_Sunday1','dia_Thursday1',
                 'dia_Tuesday1','dia_Wednesday1']], poly=poly, top=1e20, bottom=0)
print("Errores globales")
print("\tRMS: \t{}".format(rms))
print("\tMAE: \t{}".format(mae))
print("\tME: \t{}".format(me))
print("\tMPE: \t{}".format(mpe))
print("\tR2: \t{}\n".format(r2))



ranges = [[0, 5*60], [5*60, 10*60], [10*60, 20*60], [20*60, 40*60], [40*60, 60*60]]
for bottom, top in ranges:
    rms, mae, me, mpe, r2 = eval_range(p_reg, val[['Distancia_Total', 'Diferencia','Servicio','dia_habil1','dia_Monday1','dia_Saturday1','dia_Sunday1','dia_Thursday1',
                 'dia_Tuesday1','dia_Wednesday1']], poly=poly, top=top, bottom=bottom)
    print("Errores para el rango {}min a {}min".format(bottom//60, top//60))
    print("\tRMS: \t{}".format(rms))
    print("\tMAE: \t{}".format(mae))
    print("\tME: \t{}".format(me))
    print("\tMPE: \t{}".format(mpe))
    print("\tR2: \t{}\n".format(r2))

Errores globales
	RMS: 	86197.86587577837
	MAE: 	176.17042411532756
	ME: 	-8.071606549516972
	MPE: 	84.04940878290526
	R2: 	0.8763506148712268

Errores para el rango 0min a 5min
	RMS: 	6057.519070695128
	MAE: 	50.333276299306746
	ME: 	-24.90513885205872
	MPE: 	12.714068723624012
	R2: 	-0.04846875716649479

Errores para el rango 5min a 10min
	RMS: 	36197.15887710148
	MAE: 	136.16326480723595
	ME: 	-62.290904788616984
	MPE: 	36.936180009309496
	R2: 	-3.891403177349848

Errores para el rango 10min a 20min
	RMS: 	103256.37442065847
	MAE: 	235.45020030604337
	ME: 	-85.49536382113342
	MPE: 	74.97741824245497
	R2: 	-2.4906471013453535

Errores para el rango 20min a 40min
	RMS: 	201693.47323200564
	MAE: 	347.9035484866358
	ME: 	-11.207683456167945
	MPE: 	168.34793251523394
	R2: 	-0.7250785068440129

Errores para el rango 40min a 60min
	RMS: 	394230.11584134394
	MAE: 	504.9336551363482
	ME: 	424.8073123229184
	MPE: 	464.8704837296333
	R2: 	-2.3458046662384064

